In [56]:
import time
import os
import random
import pandas as pd
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from fake_useragent import UserAgent

## Configurations

In [57]:
# Configure fake user agents
ua = UserAgent()
user_agent = ua.random
options = webdriver.ChromeOptions()
options.add_argument(f'--user-agent={user_agent}')

# Configure file download path
#cwd = os.getcwd()
#prefs={"download.default_directory":cwd}
#options.add_experimental_option("prefs",prefs)

# Use headless browser
#options.add_argument("--headless=new")


## Magic numbers for criteria

In [58]:
MARKET_CAP = 2500000
PE_TTM = 30
P_CF = 15
RELATIVE_PE = .7
ROA = .1
NET_PROFIT_MARGIN = .1
ROIC = .1
CAGR = .1


## Download CSV files for all NASDAQ & NYSE listings

In [59]:
companies = {}
FINVIZ_URLS = ['https://finviz.com/screener.ashx?v=111&f=cap_microover,exch_nasd,fa_netmargin_o10,fa_roa_o10,sh_short_low&ft=4&r=','https://finviz.com/screener.ashx?v=111&f=cap_microover,exch_nyse,fa_netmargin_o10,fa_roa_o10,sh_short_low&ft=4&r=']

# Start driver
driver = webdriver.Chrome(options=options)

# Scrape NASDAQ & NYSE based on initial criteria
for url in FINVIZ_URLS:
    
    last = False
    r = 1

    # Iterate through urls until last result page 
    while not last:
        
        current_url = url + str(r)
        driver.get(current_url)
            
        html = driver.page_source
        soup = BeautifulSoup(html)

        # Add tickers to companies dict
        table = soup.find('table', class_='styled-table-new is-rounded is-tabular-nums w-full screener_table')
        for ticker in table.find_all('a', class_='tab-link'):
            if ticker.text not in companies:
                companies[ticker.text] = {}
            else:
                last = True
        r += 20

driver.quit()

## Scrape Yahoo Finance

In [74]:
YAHOO_URL_ONE = 'https://finance.yahoo.com/quote/'
YAHOO_URL_TWO = '/key-statistics'

for company in ['AAPL', 'META']:
    ua = UserAgent()
    user_agent = ua.random
    headers = {"User-Agent": user_agent}

    # Yahoo summary page
    summary_url = YAHOO_URL_ONE + company
    page = requests.get(summary_url, headers=headers, timeout=random.uniform(3, 5))
    soup = BeautifulSoup(page.content)
    market_cap = soup.find('fin-streamer',attrs={"data-field": "marketCap"}).text
    pe = soup.find('fin-streamer',attrs={"data-field": "trailingPE"}).text
    print(market_cap)
    print(pe)

    # Yahoo statistics page
    stats_url = summary_url + YAHOO_URL_TWO
    page = requests.get(summary_url, headers=headers, timeout=random.uniform(3, 5))
    soup = BeautifulSoup(page.content)

    for element in soup.find_all('td'):
        print(element)
        for e in element.findChildren():
            print(e)
            print('NEW')
    break

driver.quit()

2.908T 
29.44 
<td class="hu-tooltip-name">Date</td>
<td class="hu-tooltip-value"></td>
<td class="hu-tooltip-name">Close</td>
<td class="hu-tooltip-value"></td>
<td class="hu-tooltip-name">Open</td>
<td class="hu-tooltip-value"></td>
<td class="hu-tooltip-name">High</td>
<td class="hu-tooltip-value"></td>
<td class="hu-tooltip-name">Low</td>
<td class="hu-tooltip-value"></td>
<td class="hu-tooltip-name">Volume</td>
<td class="hu-tooltip-value"></td>
